In [36]:
%pylab inline
from glob import glob
import math
from PIL import Image
import json
import sklearn.manifold
import os

def resize_to_power_of_two(nSize):
    pos =  math.ceil(math.log2(nSize))  #(ceiling of log n with base 2)
    p   = pow(2, pos) 
    return p

def generate_spritesheet(fnames,photoSize,filename,positions,saveImage):
    """\
    Make a contact sheet from a group of filenames:

    fnames       A list of names of the image files
    
    ncols        Number of columns in the contact sheet
    nrows        Number of rows in the contact sheet
    photow       The width of the photo thumbs in pixels
    photoh       The height of the photo thumbs in pixels

    returns a PIL image object.
    """
    positionbuffer_name = 'None'
    if positions is not None:
        positionbuffer_name = filename + '.bin'
        positions.tofile(positionbuffer_name)
        dimensions = positions.shape[1]
    else:
        dimensions = 0

    # Calculate the size of the output image, based on the photo thumb sizes
    
    totalNum = len(fnames)
    
    ncols = 64
    nrows = math.ceil(totalNum/ncols)
    
    photow = photoSize[0]
    photoh = photoSize[1]
   
    
    imgWidth = ncols*photow
    imgHeight = nrows*photoh
    
    imgWidth_resized = resize_to_power_of_two(imgWidth)
    imgHeight_resized = resize_to_power_of_two(imgHeight)
    
    isize = (imgWidth_resized,imgHeight_resized) # adapt to THREE
    print('imageSize=',isize, "photow", photow, "photoh", photoh, "")

    # Create the new image. The background doesn't have to be white
    white = (0,0,0)
    inew = Image.new('RGB',isize,white)
    
    # Create JSON file to store the UV position of each image
    spriteDict = {}
    spriteDict['rows'] = nrows
    spriteDict['columns'] = ncols
    spriteDict['totalCount'] = totalNum
    spriteDict['spriteWidth'] = photow
    spriteDict['spriteHeight'] = photoh
    spriteDict['dimensions'] = dimensions
    spriteDict['bin'] = positionbuffer_name
    count = 0
    # Insert each thumb:
    for irow in range(nrows):
        for icol in range(ncols):
            left = icol*(photow)
            right = left + photow
            upper = irow*(photoh)
            lower = upper + photoh
            bbox = (left,upper,right,lower)
            try:
                # Read in an image and resize appropriately
                img = Image.open(fnames[count]).resize((photow,photoh))
            except:
                break
            inew.paste(img,bbox)
            if count not in spriteDict.keys():
                spriteDict[count] = {}
            spriteDict[count]['filename'] = os.path.basename(fnames[count])
            spriteDict[count]['uvOffset_u'] = left/imgWidth
            spriteDict[count]['uvOffset_v'] = 1.0-lower/imgHeight_resized 
            spriteDict[count]['uvRepeat_u'] = photow/imgWidth_resized
            spriteDict[count]['uvRepeat_v'] = photoh/imgHeight_resized
            count += 1
    js = json.dumps(spriteDict, indent=2)
    f = open(filename + '.json', 'w')
    f.write(js)
    f.close()
    if saveImage:
        inew.save(filename + '.png')
    return inew

Populating the interactive namespace from numpy and matplotlib


In [37]:
######Parameters Begin#########
dimensions = 3
perplexity = 500
output_corpus = 'supermarioworld'
gamedir = './pic/Super Mario World (USA)/'
runTSNE = False
######Parameters End###########

files = glob(gamedir + 'screenshots/*.png')
files = sorted(files, key=lambda x: int(os.path.basename(x)[:-4]))

if runTSNE:
    embeddings = glob(gamedir + '/embeddings/*.npy')
    embeddings = sorted(embeddings, key=lambda x: int(os.path.basename(x)[:-4]))

    length = len(embeddings)
    states = zeros((length, 256))
    for i in range(len(states)):
        states[i] = load(embeddings[i])

    tsne = sklearn.manifold.TSNE(n_components = dimensions, perplexity = perplexity)
    positions = tsne.fit_transform(states)
    print(positions.shape, end='\r')

photow,photoh = 64,56
photoSize = (photow,photoh)

spritesheet_name = './pic/' + output_corpus + str(dimensions) + '_' + str(perplexity)


In [38]:
saveImage = True
inew = generate_spritesheet(files,photoSize,spritesheet_name, positions,saveImage)
inew.show()

imageSize= (16384, 32768) photow 256 photoh 224 


In [16]:
type(positions[10][0])

numpy.float32